<a href="https://colab.research.google.com/github/tcbic/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_432_Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [1]:
!unzip forest.zip

Archive:  forest.zip
  inflating: forest/art114.jpg       
  inflating: forest/bost100.jpg      
  inflating: forest/bost101.jpg      
  inflating: forest/bost102.jpg      
  inflating: forest/bost103.jpg      
  inflating: forest/bost190.jpg      
  inflating: forest/bost98.jpg       
  inflating: forest/cdmc101.jpg      
  inflating: forest/cdmc12.jpg       
  inflating: forest/cdmc271.jpg      
  inflating: forest/cdmc277.jpg      
  inflating: forest/cdmc278.jpg      
  inflating: forest/cdmc280.jpg      
  inflating: forest/cdmc281.jpg      
  inflating: forest/cdmc282.jpg      
  inflating: forest/cdmc283.jpg      
  inflating: forest/cdmc284.jpg      
  inflating: forest/cdmc290.jpg      
  inflating: forest/cdmc291.jpg      
  inflating: forest/cdmc292.jpg      
  inflating: forest/cdmc293.jpg      
  inflating: forest/cdmc306.jpg      
  inflating: forest/cdmc313.jpg      
  inflating: forest/cdmc315.jpg      
  inflating: forest/cdmc317.jpg      
  inflating: forest/cdmc318.j

In [2]:
!unzip mountain.zip

Archive:  mountain.zip
  inflating: mountain/art1131.jpg    
  inflating: mountain/art1132.jpg    
  inflating: mountain/cdmc181.jpg    
  inflating: mountain/cdmc184.jpg    
  inflating: mountain/cdmc187.jpg    
  inflating: mountain/gre242.jpg     
  inflating: mountain/land10.jpg     
  inflating: mountain/land11.jpg     
  inflating: mountain/land13.jpg     
  inflating: mountain/land130.jpg    
  inflating: mountain/land131.jpg    
  inflating: mountain/land132.jpg    
  inflating: mountain/land142.jpg    
  inflating: mountain/land143.jpg    
  inflating: mountain/land145.jpg    
  inflating: mountain/land15.jpg     
  inflating: mountain/land150.jpg    
  inflating: mountain/land153.jpg    
  inflating: mountain/land16.jpg     
  inflating: mountain/land161.jpg    
  inflating: mountain/land165.jpg    
  inflating: mountain/land17.jpg     
  inflating: mountain/land172.jpg    
  inflating: mountain/land179.jpg    
  inflating: mountain/land18.jpg     
  inflating: mountain/land1

1) Load in image data as numpy arrays.

In [0]:
import glob, os
import cv2
import numpy as np

In [0]:
# Forest

In [0]:
# Indicate 0 for grayscale. It's the only way I could figure out for the channels when applying .shape not to appear.

os.chdir("/content/forest")
forest = []
for file in glob.glob("*.jpg"):
    image = cv2.imread(file, 0)
    forest.append(image)

In [0]:
forest_array = np.asarray(forest, dtype='float32')

In [9]:
forest_array.shape

(328, 256, 256)

In [0]:
# Save as .npy file.

np.save('forest_array', forest_array)

In [0]:
forest = np.load('forest_array.npy')

In [12]:
forest.shape

(328, 256, 256)

In [0]:
#Mountain

In [0]:
os.chdir("/content/mountain")
mountain = []
for file in glob.glob("*.jpg"):
    image = cv2.imread(file, 0)
    mountain.append(image)

In [0]:
mountain_array = np.asarray(mountain, dtype='float32')

In [15]:
mountain_array.shape

(374, 256, 256)

In [0]:
# Save as .npy file.

np.save('mountain_array', mountain_array)

In [0]:
mountain = np.load('mountain_array.npy')

In [18]:
mountain.shape

(374, 256, 256)

In [0]:
X_train = np.concatenate((forest, mountain), axis=0)

In [20]:
X_train.shape

(702, 256, 256)

In [0]:
forest_y = np.ones(len(forest))
mountain_y = np.zeros(len(mountain))
y_train = np.concatenate((forest_y, mountain_y), axis=0)

In [0]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [0]:
X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool1D, Flatten

In [0]:
# Create a model.

def create_model():
    
    model = Sequential()
    model.add(Conv2D(3, (2,2), padding='same', activation='relu', input_shape=(702, 256, 256)))
    model.add(Conv2D(3, (2,2), activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [31]:
# Instantiate model.

model = create_model()

# Define parameters.

epochs = 20
batch_size = 256

# Compile and fit.

model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

ValueError: ignored

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language